# Simulations

Go to [Index](response_Witz2019_index.ipynb).

Here we provide the script to generate the simulation files used in this response

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, copy, pickle
import numpy as np

import sys
sys.path.append(os.path.join('..','colicycle','colicycle'))
import coli_simulation as cs

In [3]:
# create variables for the path to simulation files
dir_simu = 'simulations'
if not os.path.isdir(dir_simu):
    os.makedirs(dir_simu)

## Simulations for Figure 5
The simulations run here are used in [response_Witz2019_figure5](response_Witz2019_figure5.ipynb).

In [4]:
# define conditions and list experimental parameter files
paramsdict = {}
paramsdict['glycerol'] = {}
paramsdict['glycerol']['paramfile'] = os.path.join('..', 'PreProcessed', '20180706_GW296_glycerol37_1_MMStack', 'simul_params.pkl')

paramsdict['glycerolauto'] = {}
paramsdict['glycerolauto']['paramfile'] = os.path.join('..', 'PreProcessed', '20170327_GW339_temp', 'simul_params.pkl')

paramsdict['glucose'] = {}
paramsdict['glucose']['paramfile'] = os.path.join('..', 'PreProcessed', '20180711_GW296_glucose37_1_MMStack', 'simul_params.pkl')

paramsdict['glucose8aa'] = {}
paramsdict['glucose8aa']['paramfile'] = os.path.join('..', 'PreProcessed', '20180709_GW296_glucose8aa37_1_MMStack', 'simul_params.pkl')

names = list(paramsdict.keys())
nfiles = len(names)

In [5]:
# simulation parameters
seed=123    # seed for random number generator (before each run)
numcells = [5, 500]  # N
tot_times = [200000, 2000] # T
nori_init = 1

# start loop
for ind in range(nfiles):
    ## load experimental parameters
    name = names[ind]
    paramfile = paramsdict[name]['paramfile']
    
    with open(paramfile, 'rb') as f:
        exp_params = pickle.load(f)
        
    params = {'tau_logn_mu' : exp_params['fit_logtau'][1], \
              'tau_logn_sigma' : exp_params['fit_logtau'][2], \
              'tau_corr': exp_params['tau_corr'][0], \
              'DLi_mu': exp_params['fit_DLi'][1], \
              'DLi_sigma': exp_params['fit_DLi'][2], \
              'DLdLi_logn_mu': exp_params['fit_logDLdLi'][1], \
              'DLdLi_logn_sigma': exp_params['fit_logDLdLi'][2], \
              'Lb_logn_mu': exp_params['fit_logLb'][1], \
              'Lb_logn_sigma':exp_params['fit_logLb'][2], \
              'dL_mu': exp_params['fit_dL'][1], \
              'dL_sigma': exp_params['fit_dL'][2], \
              'div_ratio': exp_params['divR_std'] \
             }

    params_bkp = {key: params[key] for key in params}
    
    ## loop on requested number of initial cells and total time
    for N,T in zip(numcells,tot_times):
        ### restart from original parameters
        params = {key: params_bkp[key] for key in params_bkp}
        
        ### 1. standard simulation -- witz et al. code
        style = 'standard'
        fname = '{:s}_{:s}_N{:d}T{:d}'.format(name, style, N, T)
        np.random.seed(seed)
        simul = cs.simul_doubleadder(N, run_time=T, params=params, name=fname, nori_init=nori_init)
        simul_pd_exp = cs.standardise_dataframe(simul)
        fpath = os.path.join(dir_simu,"{:s}.pkl".format(fname))
        simul_pd_exp.to_pickle(fpath)
        print("OUTPUT:", fpath)

        ### 2. no septum noise simulation -- witz et al. code
        style = 'symmetrical'
        params['div_ratio'] = 0.
        fname = '{:s}_{:s}_N{:d}T{:d}'.format(name, style, N, T)
        np.random.seed(seed)
        simul = cs.simul_doubleadder(N, run_time=T, params=params, name=fname, nori_init=nori_init)
        simul_pd_exp = cs.standardise_dataframe(simul)
        fpath = os.path.join(dir_simu,"{:s}.pkl".format(fname))
        simul_pd_exp.to_pickle(fpath)
        print("OUTPUT:", fpath)
        
    # end loop on modules
# end loop on parameters

OUTPUT: simulations/glycerol_standard_N5T200000.pkl
OUTPUT: simulations/glycerol_symmetrical_N5T200000.pkl
OUTPUT: simulations/glycerol_standard_N500T2000.pkl
OUTPUT: simulations/glycerol_symmetrical_N500T2000.pkl
OUTPUT: simulations/glycerolauto_standard_N5T200000.pkl
OUTPUT: simulations/glycerolauto_symmetrical_N5T200000.pkl
OUTPUT: simulations/glycerolauto_standard_N500T2000.pkl
OUTPUT: simulations/glycerolauto_symmetrical_N500T2000.pkl
OUTPUT: simulations/glucose_standard_N5T200000.pkl
OUTPUT: simulations/glucose_symmetrical_N5T200000.pkl
OUTPUT: simulations/glucose_standard_N500T2000.pkl
OUTPUT: simulations/glucose_symmetrical_N500T2000.pkl
OUTPUT: simulations/glucose8aa_standard_N5T200000.pkl
OUTPUT: simulations/glucose8aa_symmetrical_N5T200000.pkl
OUTPUT: simulations/glucose8aa_standard_N500T2000.pkl
OUTPUT: simulations/glucose8aa_symmetrical_N500T2000.pkl


### Simulations using Si & Le Treut experimental data

The experimental parameters files were constructed in [response_Witz2019_import_junlab_data](response_Witz2019_import_junlab_data.ipynb).

In [6]:
# define conditions and list experimental parameter files
names = ['mg1655_acetate', \
         'mg1655_glucose', \
         'mg1655_glycerol11aa', \
         'ncm3722_arginine', \
         'ncm3722_glucose', \
         'ncm3722_glucose12aa' \
        ]
paramsdict = {}
for name in names:
    paramsdict[name] = {}
    paramsdict[name]['paramfile'] = os.path.join('experiments', name, 'simul_params.pkl')
    
nfiles = len(names)

In [7]:
# simulation parameters
seed=123    # seed for random number generator (before each run)
numcells = [5]  # N
tot_times = [200000] # T
nori_init = 1

# start loop
for ind in range(nfiles):
    ## load experimental parameters
    name = names[ind]
    paramfile = paramsdict[name]['paramfile']
    
    with open(paramfile, 'rb') as f:
        exp_params = pickle.load(f)
        
    params = {'tau_logn_mu' : exp_params['fit_logtau'][1], \
              'tau_logn_sigma' : exp_params['fit_logtau'][2], \
              'tau_corr': exp_params['tau_corr'][0], \
              'DLi_mu': exp_params['fit_DLi'][1], \
              'DLi_sigma': exp_params['fit_DLi'][2], \
              'DLdLi_logn_mu': exp_params['fit_logDLdLi'][1], \
              'DLdLi_logn_sigma': exp_params['fit_logDLdLi'][2], \
              'Lb_logn_mu': exp_params['fit_logLb'][1], \
              'Lb_logn_sigma':exp_params['fit_logLb'][2], \
              'dL_mu': exp_params['fit_dL'][1], \
              'dL_sigma': exp_params['fit_dL'][2], \
              'div_ratio': exp_params['divR_std'] \
             }

    params_bkp = {key: params[key] for key in params}
    
    ## loop on requested number of initial cells and total time
    for N,T in zip(numcells,tot_times):
        ### restart from original parameters
        params = {key: params_bkp[key] for key in params_bkp}
        
        ### 1. standard simulation -- witz et al. code
        style = 'standard'
        fname = '{:s}_{:s}_N{:d}T{:d}'.format(name, style, N, T)
        np.random.seed(seed)
        simul = cs.simul_doubleadder(N, run_time=T, params=params, name=fname, nori_init=nori_init)
        simul_pd_exp = cs.standardise_dataframe(simul)
        fpath = os.path.join(dir_simu,"{:s}.pkl".format(fname))
        simul_pd_exp.to_pickle(fpath)
        print("OUTPUT:", fpath)

        ### 2. no septum noise simulation -- witz et al. code
        style = 'symmetrical'
        params['div_ratio'] = 0.
        fname = '{:s}_{:s}_N{:d}T{:d}'.format(name, style, N, T)
        np.random.seed(seed)
        simul = cs.simul_doubleadder(N, run_time=T, params=params, name=fname, nori_init=nori_init)
        simul_pd_exp = cs.standardise_dataframe(simul)
        fpath = os.path.join(dir_simu,"{:s}.pkl".format(fname))
        simul_pd_exp.to_pickle(fpath)
        print("OUTPUT:", fpath)
        
    # end loop on modules
# end loop on parameters

OUTPUT: simulations/mg1655_acetate_standard_N5T200000.pkl
OUTPUT: simulations/mg1655_acetate_symmetrical_N5T200000.pkl
OUTPUT: simulations/mg1655_glucose_standard_N5T200000.pkl
OUTPUT: simulations/mg1655_glucose_symmetrical_N5T200000.pkl
OUTPUT: simulations/mg1655_glycerol11aa_standard_N5T200000.pkl
OUTPUT: simulations/mg1655_glycerol11aa_symmetrical_N5T200000.pkl
OUTPUT: simulations/ncm3722_arginine_standard_N5T200000.pkl
OUTPUT: simulations/ncm3722_arginine_symmetrical_N5T200000.pkl
OUTPUT: simulations/ncm3722_glucose_standard_N5T200000.pkl
OUTPUT: simulations/ncm3722_glucose_symmetrical_N5T200000.pkl
OUTPUT: simulations/ncm3722_glucose12aa_standard_N5T200000.pkl
OUTPUT: simulations/ncm3722_glucose12aa_symmetrical_N5T200000.pkl
